In [ ]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [10]:
args_date = '2022-05-06'
src_format = '%Y-%m-%d'
src_bucket = 'xetra-1234'
target_bucket = 'xetra-1234'
columns = ["ISIN", "Date", "Time", "StartPrice", "MaxPrice", "MinPrice", "EndPrice", "TradeVolume"]
key = 'xetra_daily_report' + datetime.today().strftime("%Y-%m-%d_%H%M%S") + ".parquet"

In [ ]:
args_date_dt = datetime.strptime(args_date, src_format).date() - timedelta(days=1)

In [ ]:
# s3 resource
s3 = boto3.resource("s3")
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= args_date_dt]

In [ ]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get("Body").read().decode("utf-8")
    data = StringIO(csv_obj)
    # read the csv data
    df = pd.read_csv(data, delimiter=',')
    return df

df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

In [ ]:
df_all = df_all.loc[:, columns]

In [ ]:
df_all.dropna(inplace=True)

#### Get the Opening price per ISIN and per day

In [ ]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

#### Get the Closing price per ISIN and per day

In [ ]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

#### Aggregations

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(
    opening_price_eur=('opening_price', 'min'), 
    closing_price_eur=('closing_price', 'min'),
    minimum_price_eur=('MinPrice', 'min'),
    maximum_price_eur=('MaxPrice', 'max'),
    daily_trader_volume=('TradedVolume', 'sum')
)

#### Percent change Previous Closing

In [ ]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [ ]:
df_all['change_prev_closing_%'] = (
    df_all['closing_price_eur'] - df_all['prev_closing_price']
) / df_all['prev_closing_price'] * 100

In [ ]:
df_all.drop(
    columns=['prev_closing_price'],
    inplace=True
)

In [ ]:
df_all = df_all.round(decimals=2)

In [ ]:
df_all = df_all[df_all.Date >= args_date]

#### Write to S3

In [ ]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)

# Target bucket instance
bucket_target = s3.Bucket(target_bucket)
bucket_target.put_object(Body=out_buffer.get_value(), key=key)

#### Reading the uploaded file

In [ ]:
for obj in bucket_target.objects.all():
    print(obj.key)

In [ ]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20230510_101629.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)